In [1]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 123 kB 934 kB/s eta 0:00:01
     |████████████████████████████████| 128 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 675 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 547 kB 9.7 MB/s eta 0:00:01
  Attempting uninstall: docutils
    Found existing installation: docutils 0.16
    Uninstalling docutils-0.16:
      Successfully uninstalled docutils-0.16


In [1]:
import pandas as pd
import numpy as np


import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import time

In [2]:
torch.cuda.set_device(1)

COLS = ['Text_tokens', 'Tweet_id']
file_path = '/home/kim2/bert_example'
file_name = '/bert_example_file'

In [3]:
# Store the model we want to use
MODEL_NAME = "bert-base-multilingual-cased"

# We need to create the model and tokenizer
model = BertModel.from_pretrained(MODEL_NAME)
# tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = model.to('cuda')

In [4]:
## 예시로 1000개만 불러옴
df_chucks = pd.read_csv(file_path+file_name+'.csv', chunksize=1000, skiprows=1, names=COLS)
df = df_chucks.get_chunk()

In [5]:
## int형 리스트로 반환
def int_list2(something):
    something = something.split('\t')
    return [int(x) for x in something]

tmp_list = df['Text_tokens'].apply(int_list2)

In [6]:
## 각 list를 넣으면 pooled로 768크기의 임베딩 반환
i = 0
outputs, pooled = model(torch.tensor([tmp_list[i]]).to('cuda'))
pooled.size()

torch.Size([1, 768])

In [7]:
pooled

tensor([[ 2.8400e-01, -1.2304e-01,  2.3435e-01, -1.1316e-01,  1.1526e-01,
          2.3079e-01,  1.4954e-01,  3.8077e-01, -3.9197e-01,  2.2551e-01,
          5.8694e-02, -1.4300e-01, -1.9126e-01, -1.2691e-01,  2.5607e-01,
         -1.8712e-01,  5.7559e-01,  4.4582e-02,  1.7263e-03, -2.7687e-01,
         -9.9998e-01, -7.7590e-02, -4.2319e-01, -1.2326e-01, -5.5288e-01,
          1.2421e-01, -2.2944e-01,  1.4024e-01,  2.1159e-01, -5.1719e-02,
          9.7446e-02, -9.9998e-01,  1.6962e-01,  6.2137e-01,  1.5128e-01,
         -1.0012e-01,  1.8383e-01,  4.2255e-04,  9.8842e-02, -3.4832e-01,
         -1.5683e-01,  1.1988e-01, -1.1113e-02,  1.7970e-01, -1.2302e-01,
         -1.7984e-01, -2.9587e-01,  2.7734e-03, -2.2120e-01,  6.3745e-03,
          9.3762e-02,  3.9179e-02,  2.8411e-01,  3.5141e-01,  2.4317e-01,
          1.4233e-01,  4.1567e-01, -8.4730e-02,  2.6743e-01,  4.3052e-02,
          9.5099e-02,  2.4541e-01,  1.1238e-01, -7.8607e-02, -3.0730e-01,
         -2.9500e-01,  8.2424e-02, -2.

In [8]:
## tmp_df에 768 columns로 append하고 Tweet_id를 추가

start = time.time()  # 시작 시간 저장


tmp_df = pd.DataFrame()
for i in range(len(tmp_list)):
    outputs, pooled = model(torch.tensor([tmp_list[i]]).to('cuda'))
    tmp_df = tmp_df.append(pd.DataFrame(pooled.to('cpu').detach().numpy()))

tmp_df['Tweet_id'] = df['Tweet_id'].values


print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

time : 23.123438596725464


In [9]:
tmp_df.shape

(1000, 769)

In [10]:
tmp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,759,760,761,762,763,764,765,766,767,Tweet_id
0,0.155626,-0.012765,0.333185,-0.220910,-0.018613,0.107525,0.145422,0.232999,-0.323440,0.134869,...,0.255099,0.037119,0.551091,0.304010,0.028520,0.149649,-0.096914,0.111290,0.118417,1B4DF7004EEFA36DA0FDD5B57E243B01
0,0.221007,0.061440,0.286726,-0.109970,-0.015690,0.462348,0.089962,0.184447,-0.267669,0.153645,...,0.280598,0.167128,0.655303,0.347380,0.336593,0.188523,-0.260219,0.183545,0.024493,AB1873002F780C1BEB6428A80B43D48C
0,0.151381,-0.144505,0.236894,-0.085393,-0.026727,0.139243,0.131343,0.183699,-0.215228,0.071623,...,0.224680,0.128973,0.393332,0.238875,0.018895,0.052683,-0.185170,0.034306,0.048411,3EFF4526AE0159669433C5C934A82B76
0,0.348671,-0.014143,0.248030,-0.165740,0.016267,0.273950,0.267105,0.144006,-0.396443,0.237640,...,0.150975,0.286068,0.727432,0.279369,0.228796,0.101203,-0.296461,0.311506,0.046299,C0654B3F48FEA0722571F70B6E0FB956
0,0.261870,-0.092498,0.388850,-0.228731,-0.136980,0.514632,0.206241,0.363983,-0.438781,0.345100,...,0.445574,0.023426,0.841468,0.353072,0.286669,0.373736,-0.306973,0.090443,0.143095,AE9DD310CF14AFD4F1A95902521FF132
